In [1]:
import os
from Custom_func import initialize_directory_structure, split_and_prepare_Yolo_data,generate_yaml_config

In [2]:
# Example usage
if __name__ == "__main__":
    classes =['Masked', 'Unmasked', 'Improperly_Masked']
    base_dir = os.getcwd()  # Base path for saving the dataset

    # Initialize the dataset folder structure
    initialize_directory_structure(base_dir)

    # List of image files
    img_folder = r'Dataset\images'
    annot_folder = r'Dataset\annotations'
    image_files = [file for file in os.listdir(img_folder) if file.endswith(('.jpg', '.jpeg', '.png'))]

    # Process the dataset
    split_and_prepare_Yolo_data(image_files, img_folder, annot_folder, base_dir, classes)

    # Generate the YAML configuration file
    generate_yaml_config(base_dir, classes)